In [2]:
import os, sys, logging, importlib, math
import rasterio, affine

import geopandas as gpd
import pandas as pd

from zipfile import ZipFile
from affine import Affine
from rasterio import features
from rasterio.mask import mask
from rasterio.features import rasterize

from GEP import gep_results

# Mapping electrification
We need to convert certain fields in the results to a raster; these fields need to be calculated from the existing data

1. Population already electrified 2030
    a. Rasterize field **ElecPop**
2. New population electrified by grid 2030
    a. Filter data where **FinalElecCode2030** == 1.0
    b. Set **NewConnections2025** = 0 where **FinalElecCode2025** == 99.0
    c. Combine **NewConnections2030** with **NewConnections2025**
3. New population electrified by off grid 2030
    a. Filter data where **FinalElecCode2030** != 1.0
    b. Set **NewConnections2025** = 0 where **FinalElecCode2025** != **FinalElecCode2030**
    c. Combine **NewConnections2030** with **NewConnections2025**

In [3]:
# Identify countries to process
clustersFolder = '/media/gost/DATA1/GEP/Clusters'
zipFiles = []
processedCountries = []
for root, folders, files in os.walk(clustersFolder):    
    processedCountries.append([f for f in folders])
    for f in files:
        if f[-4:] == ".zip":
            zipFiles.append(f[:-4])
            
processedCountries = [item for sublist in processedCountries for item in sublist]

In [10]:
res['Pop2030'].min()

5.2194929122924805

In [11]:
print(res.shape)
res2 = res.loc[res['Pop2030'] > 10,]
print(res2.shape)

(300503, 129)
(270215, 129)


In [13]:
import GEP
importlib.reload(GEP)
from GEP import gep_results

resolution = 0.01
scenario = "0_1_0_0_0_0"
elecRate = 0.9

for country in processedCountries:
    print(f"Processing {country}")
    #try:
    xx_so = gep_results(country, 
                scenariosFolder='/media/gost/DATA1/GEP/Scenario_0_1_0_0_0_0',
                clustersFolder = '/media/gost/DATA1/GEP/Clusters',
                scenario=scenario)
    outFile = os.path.join(xx_so.countryClustersFolder, "rasterized_gep_%s_%s_gt10.tif" % (resolution, scenario))
    res = xx_so.join_results()
    res = res.loc[res['Pop2030'] > 10,]
    res['already_electrified'] = (res['ElecPop'] / res['Pop']) > elecRate
    res.loc[res['already_electrified'] == 1, 'FinalElecCode2030'] = 99
    xx_so.rasterize_results(res, outFile, res=resolution)
    #except:
    #    print(f'error processing {country}')

Processing ao-1
Processing bd-1
Processing bf-1
Processing bi-1
Processing bj-1
Processing bw-1
Processing cd-1
Processing gw-1
Processing hn-1
Processing ht-1
Processing ke-1
Processing kh-1
Processing km-1
Processing lr-1
Processing ls-1
Processing mg-1
Processing ml-1
Processing mm-1
Processing mn-1
Processing mr-1
Processing mw-1
Processing mz-1
Processing na-1
Processing ne-1
Processing ni-1
Processing pg-1
Processing pk-1
Processing rw-1
Processing sb-1


/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/pandas/core/reshape/merge.py:946: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)


Processing sd-1
Processing sl-1
Processing sn-1
Processing cg-1
Processing ci-1
Processing cm-1
Processing dj-1
Processing er-1
Processing et-1
Processing fm-1
Processing gh-1
Processing gm-1
Processing gn-1
Processing cf-1
Processing ga-1
Processing gq-1
Processing ng-1
Processing so-1
Processing ss-1
Processing st-1
Processing sz-1
Processing td-1
Processing tg-1
Processing tl-1
Processing tz-1
Processing ug-1
Processing vu-1
Processing za-1


/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Processing zm-1
Processing zw-1


In [ ]:
# Convert 0 in data to no data
allImages = []
for root, folders, files in os.walk(clustersFolder):
    for f in files:
        if "%s_%s.tif" % (resolution, scenario) in f:
            allImages.append(os.path.join(root, f))

for inFile in allImages:
    outFile = inFile.replace(".tif", "_noData.tif")
    print(f'gdal_translate -of GTiff -a_nodata 0 {inFile} {outFile}')

In [ ]:
print(f'find /media/gost/DATA/GEP/Clusters -name "*{resolution}_{scenario}_noData.tif" > tifFiles.txt')
print(f"gdalbuildvrt combined_gep_rasterized_{resolution}_{scenario}.vrt -input_file_list tifFiles.txt")